In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("LBW_Dataset.csv")
df.sample(10)

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence,Result
35,4,26.0,65.0,1.0,9.0,1,1.375000,5.0,2.0,1
40,1,24.0,40.0,1.0,8.9,1,1.714000,5.0,1.0,1
58,1,26.0,60.0,1.0,9.2,0,1.714286,5.0,1.0,1
29,1,28.0,60.0,1.0,9.0,1,1.375000,5.0,1.0,1
26,1,24.0,40.0,1.0,9.5,1,1.714286,5.0,1.0,1
91,3,21.0,55.0,1.0,9.0,0,1.375000,5.0,1.0,1
88,3,21.0,48.0,1.0,NaN,0,NaN,5.0,1.0,1
27,1,26.0,45.0,1.0,9.0,1,1.375000,5.0,1.0,1
45,3,25.0,45.0,1.0,9.3,1,1.375000,5.0,1.0,1
75,1,24.0,43.0,1.0,9.0,1,1.666667,5.0,2.0,1


In [3]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [4]:
print(df.shape)

(64, 10)


In [5]:
df.isna().sum()

Community         0
Age               0
Weight            0
Delivery phase    0
HB                0
IFA               0
BP                0
Education         0
Residence         0
Result            0
dtype: int64

In [6]:
df.dtypes

Community           int64
Age               float64
Weight            float64
Delivery phase    float64
HB                float64
IFA                 int64
BP                float64
Education         float64
Residence         float64
Result              int64
dtype: object

In [38]:
class FCLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size + output_size)
        self.bias = np.random.randn(1, output_size) / np.sqrt(input_size + output_size)
        

    def forward_propagation(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # bias_error = output_error
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error
    
    
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward_propagation(self, input):
        self.input = input
        return self.activation(input)
    
    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)
    
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

class NN:

    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
    
    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    
    ''' X and Y are dataframes '''
    
    def fit(self,X,Y,epochs,learning_rate):
        '''
        Function that trains the neural network by taking x_train and y_train samples as input
        '''
        # sample dimension first
        X = X.to_numpy().reshape((X.shape[0],X.shape[1],1))
        Y = Y.to_numpy().reshape((Y.shape[0],1))
        
        samples = len(X)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = X[j]
                output = output.transpose()
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(Y[j], output)

                # backward propagation
                error = self.loss_prime(Y[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            # print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
    
    def predict(self,X):

        """
        The predict function performs a simple feed forward of weights
        and outputs yhat values 

        yhat is a list of the predicted value for df X
        """
        
        # sample dimension first
        X = X.to_numpy().reshape((X.shape[0],X.shape[1],1))
        samples = len(X)
        yhat = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = X[i]
            output = output.transpose()
            for layer in self.layers:
                output = layer.forward_propagation(output)
            yhat.append(output)
        
        return yhat

    def CM(self,y_test,y_test_obs):
        '''
        Prints confusion matrix 
        y_test is list of y values in the test dataset
        y_test_obs is list of y values predicted by the model

        '''
        y_test = y_test.to_numpy().reshape((y_test.shape[0],1))
        for i in range(len(y_test_obs)):
            if(y_test_obs[i]>0.6):
                y_test_obs[i]=1
            else:
                y_test_obs[i]=0
        
        cm=[[0,0],[0,0]]
        fp=0
        fn=0
        tp=0
        tn=0
        
        for i in range(len(y_test)):
            if(y_test[i]==1 and y_test_obs[i]==1):
                tp=tp+1
            if(y_test[i]==0 and y_test_obs[i]==0):
                tn=tn+1
            if(y_test[i]==1 and y_test_obs[i]==0):
                fp=fp+1
            if(y_test[i]==0 and y_test_obs[i]==1):
                fn=fn+1
        cm[0][0]=tn
        cm[0][1]=fp
        cm[1][0]=fn
        cm[1][1]=tp

        a= (tp+tn)/(tp+tn+fp+fn)
        p= tp/(tp+fp)
        r=tp/(tp+fn)
        f1=(2*p*r)/(p+r)
        
        print("Confusion Matrix : ")
        print(cm)
        print("\n")
        print(f"Accuracy : {a}")
        print(f"Precision : {p}")
        print(f"Recall : {r}")
        print(f"F1 SCORE : {f1}")

In [46]:
for i in range(50):
    print("Iterations Number = ",i+1)
    X = df.iloc[:, :-1]
    y = df.iloc[:,-1]
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    # print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



    net = NN()
    net.add(FCLayer(9,5))
    net.add(ActivationLayer(tanh,tanh_prime))
    net.add(FCLayer(5,1))
    net.add(ActivationLayer(sigmoid,sigmoid_prime))

    # train
    net.use(mse, mse_prime)
    net.fit(X_train, y_train, epochs=30, learning_rate=0.1)

    # test
    out = net.predict(X_train)
    # print(out)

    net.CM(y_test,out)
    print("\n\n")

Iterations Number =  1
Confusion Matrix : 
[[0, 0], [2, 20]]


Accuracy : 0.9090909090909091
Precision : 1.0
Recall : 0.9090909090909091
F1 SCORE : 0.9523809523809523



Iterations Number =  2
Confusion Matrix : 
[[0, 0], [3, 19]]


Accuracy : 0.8636363636363636
Precision : 1.0
Recall : 0.8636363636363636
F1 SCORE : 0.9268292682926829



Iterations Number =  3
Confusion Matrix : 
[[0, 0], [4, 18]]


Accuracy : 0.8181818181818182
Precision : 1.0
Recall : 0.8181818181818182
F1 SCORE : 0.9



Iterations Number =  4
Confusion Matrix : 
[[0, 0], [1, 21]]


Accuracy : 0.9545454545454546
Precision : 1.0
Recall : 0.9545454545454546
F1 SCORE : 0.9767441860465117



Iterations Number =  5
Confusion Matrix : 
[[0, 0], [1, 21]]


Accuracy : 0.9545454545454546
Precision : 1.0
Recall : 0.9545454545454546
F1 SCORE : 0.9767441860465117



Iterations Number =  6
Confusion Matrix : 
[[0, 0], [4, 18]]


Accuracy : 0.8181818181818182
Precision : 1.0
Recall : 0.8181818181818182
F1 SCORE : 0.9



Iterations

Confusion Matrix : 
[[0, 0], [1, 21]]


Accuracy : 0.9545454545454546
Precision : 1.0
Recall : 0.9545454545454546
F1 SCORE : 0.9767441860465117



